# Evolver Loop 3 Analysis

## Goal: Verify overlap validation and plan next steps

The evaluator claims:
1. Current submission has 45 groups with strict overlaps using `intersects() and not touches()`
2. The executor used `intersection.area > 0` which is NOT the same as the LB check
3. Only chistyakov_best.csv is truly valid

Let me verify this and understand the situation.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Tree geometry
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Functions defined')

Functions defined


In [2]:
# Define BOTH overlap checks
def check_overlap_area_based(polygons):
    """Executor's check: intersection.area > 0"""
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            intersection = polygons[i].intersection(polygons[j])
            if intersection.area > 0:
                return True, (i, j), intersection.area
    return False, None, 0

def check_overlap_lb_style(polygons):
    """LB's check: intersects() and not touches()"""
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True, (i, j)
    return False, None

print('Overlap checks defined')

Overlap checks defined


In [3]:
# Load current submission and check with BOTH methods
print('Loading current submission...')
current_df = pd.read_csv('/home/submission/submission.csv')
current_parsed = parse_submission(current_df)
print(f'Shape: {current_parsed.shape}')

# Check with both methods
area_overlaps = []
lb_overlaps = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    # Area-based check (executor's method)
    has_area_overlap, _, _ = check_overlap_area_based(polygons)
    if has_area_overlap:
        area_overlaps.append(n)
    
    # LB-style check
    has_lb_overlap, _ = check_overlap_lb_style(polygons)
    if has_lb_overlap:
        lb_overlaps.append(n)

print(f'\nArea-based overlaps (executor method): {len(area_overlaps)}')
print(f'LB-style overlaps (intersects and not touches): {len(lb_overlaps)}')

if lb_overlaps:
    print(f'\nLB-style overlap configs: {lb_overlaps[:20]}...')

Loading current submission...
Shape: (20100, 4)



Area-based overlaps (executor method): 32
LB-style overlaps (intersects and not touches): 34

LB-style overlap configs: [8, 17, 19, 26, 27, 29, 30, 33, 40, 41, 42, 46, 48, 50, 53, 61, 66, 69, 71, 75]...


In [ ]:
# Check all preoptimized files with LB-style validation
print('Checking preoptimized files with LB-style validation...')
print('='*60)

files = [
    'best_snapshot.csv',
    'better_ensemble.csv',
    'bucket_of_chump.csv',
    'chistyakov_best.csv',
    'ensemble_70_627.csv',
    'saspav_best.csv',
    'submission_70_926.csv',
]

def calculate_score(df):
    total_score = 0.0
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        config_df = df[df['id'].str.startswith(prefix)]
        if len(config_df) != n:
            continue
        polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
        union = unary_union(polygons)
        bounds = union.bounds
        side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
        total_score += (side ** 2) / n
    return total_score

file_stats = {}
for filename in files:
    try:
        df = pd.read_csv(f'/home/code/preoptimized/{filename}')
        parsed = parse_submission(df)
        score = calculate_score(parsed)
        
        # Check LB-style overlaps
        lb_overlaps = 0
        for n in range(1, 201):
            prefix = f'{n:03d}_'
            config_df = parsed[parsed['id'].str.startswith(prefix)]
            if len(config_df) != n:
                continue
            polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
            has_overlap, _ = check_overlap_lb_style(polygons)
            if has_overlap:
                lb_overlaps += 1
        
        file_stats[filename] = {'score': score, 'lb_overlaps': lb_overlaps, 'df': parsed}
        status = 'VALID' if lb_overlaps == 0 else f'{lb_overlaps} overlaps'
        print(f'{filename:25s}: {score:.6f} ({status})')
    except Exception as e:
        print(f'{filename:25s}: ERROR - {e}')

print('='*60)

In [ ]:
# Create a TRULY valid ensemble using LB-style validation
print('\nCreating TRULY valid ensemble with LB-style validation...')

best_configs = {}
best_sources = {}
best_sides = {}

for n in range(1, 201):
    prefix = f'{n:03d}_'
    best_side = float('inf')
    best_config = None
    best_source = None
    
    for filename, data in file_stats.items():
        df = data['df']
        config_df = df[df['id'].str.startswith(prefix)].copy()
        
        if len(config_df) != n:
            continue
        
        # Create polygons and check LB-style overlaps
        polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
        has_overlap, _ = check_overlap_lb_style(polygons)
        
        if not has_overlap:
            union = unary_union(polygons)
            bounds = union.bounds
            side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
            if side < best_side:
                best_side = side
                best_config = config_df
                best_source = filename
    
    if best_config is not None:
        best_configs[n] = best_config
        best_sources[n] = best_source
        best_sides[n] = best_side
    else:
        print(f'WARNING: No valid config found for N={n}')

print(f'Found valid configs for {len(best_configs)} out of 200 N values')

In [ ]:
# Build and verify the truly valid ensemble
print('Building truly valid ensemble...')

ensemble_data = []
for n in range(1, 201):
    if n in best_configs:
        for _, row in best_configs[n].iterrows():
            ensemble_data.append({
                'id': row['id'],
                'x': row['x'],
                'y': row['y'],
                'deg': row['deg']
            })

valid_ensemble = pd.DataFrame(ensemble_data)
print(f'Ensemble shape: {valid_ensemble.shape}')

# Calculate score
valid_score = sum((best_sides[n] ** 2) / n for n in best_sides)
print(f'Valid ensemble score: {valid_score:.6f}')

# Final verification with LB-style check
print('\nFinal LB-style verification...')
final_overlaps = 0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = valid_ensemble[valid_ensemble['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    has_overlap, _ = check_overlap_lb_style(polygons)
    if has_overlap:
        final_overlaps += 1
        print(f'N={n}: has overlap!')

if final_overlaps == 0:
    print('NO LB-STYLE OVERLAPS! This should pass LB validation.')
else:
    print(f'Found {final_overlaps} configs with LB-style overlaps')

In [ ]:
# Source breakdown
from collections import Counter
print('\nSource breakdown for valid ensemble:')
source_counts = Counter(best_sources.values())
for source, count in source_counts.most_common():
    print(f'{source}: {count} configs')

In [ ]:
# Save the truly valid ensemble
print('\nSaving truly valid ensemble...')

import os
os.makedirs('/home/submission', exist_ok=True)

# Format with 's' prefix
submission = valid_ensemble.copy()
for col in ['x', 'y', 'deg']:
    submission[col] = 's' + submission[col].apply(lambda x: f'{x:.15f}')

submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/003_strict_ensemble/truly_valid_submission.csv', index=False)

print('Submission saved!')
print(f'Score: {valid_score:.6f}')
print(f'Target: 68.897509')
print(f'Gap: {valid_score - 68.897509:.6f}')